In [ ]:
# Stocks Symbol List Data Source: https://datahub.io/core/nyse-other-listings#python
# Stock Data Source: FinnHub

In [1]:
# packages
# data
import finnhub
from datapackage import Package
import yfinance as yf

# scientific
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# utils
import time
import requests
import json



In [ ]:
################################################################################################

In [ ]:
# Gathering data from https://datahub.io/core/nyse-other-listings#python 
# -- on 01.12.2021 8497 symbols were captured
# stocks = []
# package = Package('https://datahub.io/core/nyse-other-listings/datapackage.json')

# # print list of all resources:
# # print(package.resource_names)

# # print processed tabular data (if exists any)
# for resource in package.resources:
#     if resource.descriptor['datahub']['type'] == 'derived/csv':
#         for entry in resource.read():
#             stocks.append(entry[0])

In [2]:
# EXTRA SYMBOLS
weed_stocks = ["GRWG", "JUSHF", "CRLBF", "HRVSF", "MJ", "CURLF", "CGC", "CRON", "MEDIF"]

tech_stocks = ["SNE", "XRX", "CSCO", "SIRI", "ROKU", "ATVI", "EA", "VZ", "T", "TMUS", "AMAT",
              "TXN", "MU", "FB", "MSFT", "SNAP", "AMZN", "TWTR", "AMD", "QCOM", "AVGO", "AAPL",
              "INTC", "NVDA", "NFLX", "ORCL", "IBM"]

fin_stocks = ["AXP", "V", "MA", "JPM", "DFS", "MS", "BAC", "GS", "SQ", "PYPL", "GDOT", "AMP"]

energy_stocks = ["NGG", "PCG", "RDS-B", "OXY", "MRO", "DD", "CVX", "DOW", "XOM", "PSX", "COP", 
              "HAL", "PLUG"]

travel_stocks = ["LUV", "MAR", "ABNB", "CCL", "RCL", "DIS", "AAL", "JBLU", "SAVE", "DAL", "HLT"]

food_consumer_stocks = ["DRI", "GIS", "NKE", "PG", "SHAK", "GRUB", "KDP", "PEP", "KO", "K", "QSR", "CMG",
              "YUM", "BGS", "ACI", "DAR", "KR", "GE", "GOOS", "HD", "LOW", "LULU", "DPZ", "SBUX", "MMM",
              "WMT", "TGT", "COST", "JACK", "MCD", "UBER", "GME"]

pharma_stocks = ["CVS", "RAD", "UNH", "LLY", "JNJ", "BMY", "AZN", "MRK", "ABBV", "GILD", "PFE"]

defense_infra_stocks = ["GD", "LMT", "NOC", "BA", "CAT", "WM"]

auto_stocks = ["HMC", "TM", "F", "TSLA", "GM"]

misc_stocks = ["ISR"]

all_stocks = (weed_stocks + tech_stocks + fin_stocks + energy_stocks 
                + travel_stocks + food_consumer_stocks + pharma_stocks 
                + defense_infra_stocks + auto_stocks + misc_stocks)

# pd.DataFrame(all_stocks).to_csv('./data/all_stocks.csv', index=False)

# excluding pharma
# all_stocks = (weed_stocks + tech_stocks + fin_stocks + energy_stocks 
#               + travel_stocks + food_consumer_stocks + defense_infra_stocks 
#               + auto_stocks + misc_stocks)

# for symb in all_stocks:
#     if str(symb) not in stocks:
#         stocks.append(str(symb))

In [ ]:
# scrubbing data of symbols which contain bad_chars = ["$", ";", ":", "!", "*", "."]
# -- run until there is no more output
# bad_chars = ["$", ";", ":", "!", "*", "."]

# for s in stocks:
#     for char in bad_chars:
#         if char in str(s):
#             print(s)
#             stocks.remove(str(s))

In [3]:
display(len(all_stocks))

127

In [ ]:
################################################################################################

In [4]:
df = {}

In [5]:
# Adding company beta and 10_day_avg_volume values to a dataframe
count = 1
repeat = []

# Stock data
for stock in all_stocks:
# for stock in ["ABNB", ]:
    try:
        # request company's beta
        r = requests.get('https://finnhub.io/api/v1/stock/metric?symbol=' + str(stock) + '&metric=all&token=bvt5fmf48v6rku8bvmn0')
        metrics = r.json()["metric"]
        count += 1

        beta = metrics["beta"]
        vol = metrics["10DayAverageTradingVolume"]
        if beta == None or vol == None or (str(stock) in df.keys()):
            continue

        # fill dictionary
        if beta >= 1.0:
            df[str(stock)] = {"beta": beta, "volume_10_day": vol}

    except:
        display(r.json())
        display(stock)
        repeat.append(stock)
        time.sleep(30)
        pass
    
for stock in repeat:
    try:
        # request company's beta
        r = requests.get('https://finnhub.io/api/v1/stock/metric?symbol=' + str(stock) + '&metric=all&token=bvt5fmf48v6rku8bvmn0')
        metrics = r.json()["metric"]
        count += 1

        beta = metrics["beta"]
        vol = metrics["10DayAverageTradingVolume"]
        if beta == None or vol == None or (str(stock) in df.keys()):
            continue

        # fill dictionary
        if beta >= 1.0:
            df[str(stock)] = {"beta": beta, "volume_10_day": vol}

    except:
        display(r.json())
        display(stock)
        time.sleep(30)
        pass

{'metric': {}, 'metricType': 'all', 'series': {}, 'symbol': 'MJ'}

'MJ'

{'error': 'API limit reached. Please try again later. Remaining Limit: 0'}

'PLUG'

{'error': 'API limit reached. Please try again later. Remaining Limit: 0'}

'HMC'

{'error': 'API limit reached. Please try again later. Remaining Limit: 0'}

'TM'

{'metric': {}, 'metricType': 'all', 'series': {}, 'symbol': 'MJ'}

'MJ'

In [6]:
# dataframe to store stock data
data = pd.DataFrame(df)
data = data.transpose()
display(data)

,beta,volume_10_day
GRWG,3.11377,2.53634
CURLF,1.57790,0.97750
CGC,2.37242,1.44393
CRON,2.11014,1.05673
XRX,1.87977,2.58729
...,...,...
F,1.27192,125.95161
TSLA,2.13589,34.20522
GM,1.38764,38.65255
ISR,2.27569,19.66579


In [7]:
data['c'] = ''
data['o'] = ''
data['adx'] = ''
data['signal'] = ''
data['trending'] = ''
data['shareOutstanding'] = ''

In [8]:
# adding trends, signals, and price levels
count = 1
repeat = []

# Stock data
# for stock in ["MA", "CURLF", "AXP", "COP", "HAL", "DAR", "GOOS"]:
for stock in data.index:
    try:
        # Aggregate Indicators -- (signal, adx, trending)
        r = requests.get('https://finnhub.io/api/v1/scan/technical-indicator?symbol=' + str(stock) + '&resolution=W&token=bvt5fmf48v6rku8bvmn0')
        metrics = r.json()
        count += 1

        # trending metrics
        signal = metrics["technicalAnalysis"]["signal"]
        adx = metrics["trend"]["adx"]
        trending = metrics["trend"]["trending"]
        # add to dataframe
        data.at[str(stock), 'adx'] = adx
        data.at[str(stock), 'signal'] = signal
        data.at[str(stock), 'trending'] = trending


        # company profile -- shareOutstanding 
        r = requests.get('https://finnhub.io/api/v1/stock/profile2?symbol=' + str(stock) + '&token=bvt5fmf48v6rku8bvmn0')
        shares = r.json()["shareOutstanding"]
        # add to dataframe
        data.at[str(stock), 'shareOutstanding'] = shares

        # open price and current price
        r = requests.get('https://finnhub.io/api/v1/quote?symbol=' + str(stock) + '&token=bvt5fmf48v6rku8bvmn0')
        metrics = r.json()
        # add to dataframe
        data.at[str(stock), 'o'] = metrics['o']
        data.at[str(stock), 'c'] = metrics['c']

        # News Sentiment
        r = requests.get('https://finnhub.io/api/v1/news-sentiment?symbol=' + str(stock) + '&token=bvt5fmf48v6rku8bvmn0')
        metrics = r.json()
        # add to dataframe
        data.at[str(stock), 'bearish'] = metrics["sentiment"]["bearishPercent"]
        data.at[str(stock), 'bullish'] = metrics["sentiment"]["bullishPercent"]

    except:
        print(str(count) + '...' + str(stock))
        time.sleep(30)
        repeat.append(stock)
        pass
    
print("repeating: " + str(repeat))
for stock in repeat:
    try:
        # Aggregate Indicators -- (signal, adx, trending)
        r = requests.get('https://finnhub.io/api/v1/scan/technical-indicator?symbol=' + str(stock) + '&resolution=W&token=bvt5fmf48v6rku8bvmn0')
        metrics = r.json()
        count += 1

        signal = metrics["technicalAnalysis"]["signal"]
        adx = metrics["trend"]["adx"]
        trending = metrics["trend"]["trending"]

        # add to dataframe
        data.at[str(stock), 'adx'] = adx
        data.at[str(stock), 'signal'] = signal
        data.at[str(stock), 'trending'] = trending


        # company profile -- shareOutstanding 
        r = requests.get('https://finnhub.io/api/v1/stock/profile2?symbol=' + str(stock) + '&token=bvt5fmf48v6rku8bvmn0')
        shares = r.json()["shareOutstanding"]

        # add to dataframe
        data.at[str(stock), 'shareOutstanding'] = shares


        # open price and current price
        r = requests.get('https://finnhub.io/api/v1/quote?symbol=' + str(stock) + '&token=bvt5fmf48v6rku8bvmn0')
        metrics = r.json()

        # add to dataframe
        data.at[str(stock), 'o'] = metrics['o']
        data.at[str(stock), 'c'] = metrics['c']

        # News Sentiment
        r = requests.get('https://finnhub.io/api/v1/news-sentiment?symbol=' + str(stock) + '&token=bvt5fmf48v6rku8bvmn0')
        metrics = r.json()

        data.at[str(stock), 'bearish'] = metrics["sentiment"]["bearishPercent"]
        data.at[str(stock), 'bullish'] = metrics["sentiment"]["bullishPercent"]

    except:
        print(str(count) + '...' + str(stock))
        time.sleep(30)
        pass

5...CRON
17...AVGO
33...OXY
34...MRO
50...HLT
51...DRI
67...F
68...TSLA
70...ISR
repeating: ['CRON', 'AVGO', 'OXY', 'MRO', 'HLT', 'DRI', 'F', 'TSLA', 'ISR']
72...CRON
80...ISR


In [9]:
# data = data.sort_values(by=["signal", "beta", "shareOutstanding"], axis=0, ascending=False)
data = data.sort_values(by=["adx", "shareOutstanding"], axis=0, ascending=False)
# data.to_csv('./data/01_25_31_2132.csv', index=True)
# data = data.drop("RDS-B", axis=0)
n = len(data)
step = 15

data_1 = data[:step]
data_2 = data[step:step*2]
data_3 = data[step*2:step*3]
data_4 = data[step*3:]

ls = [data_1, data_2, data_3, data_4]

for df in ls:
    display(df)
#     break

,beta,volume_10_day,c,o,adx,signal,trending,shareOutstanding,bearish,bullish
CRON,2.11014,1.05673,,,,,,,NaN,NaN
GRWG,3.11377,2.53634,43.2299,42.365,70.9655,buy,True,36.88,0.0000,1.0000
SNAP,1.29450,18.94993,51.8,51.3035,63.3651,buy,True,1415.8,0.0000,1.0000
PLUG,1.87740,73.78250,63.365,63.5101,61.9773,buy,True,303.38,0.5000,0.5000
GME,1.73451,67.29940,256.89,282,61.2611,buy,True,64.3,0.6327,0.3673
DAR,1.27059,1.49727,62.41,62.03,57.4987,buy,True,163.78,0.5000,0.5000
TSLA,2.13589,34.20522,792.64,784.05,56.6435,buy,True,947.9,0.5676,0.4324
ROKU,1.94542,4.12447,388.025,384.63,53.8478,buy,True,119.9,0.0000,1.0000
SHAK,1.78637,1.17170,112.62,112.74,50.6535,buy,True,37.56,0.5000,0.5000
NVDA,1.39993,7.35356,519.33,518.395,46.8965,neutral,True,613,0.0000,1.0000


,beta,volume_10_day,c,o,adx,signal,trending,shareOutstanding,bearish,bullish
TXN,1.06721,4.13966,165.47,165.38,39.9076,buy,True,919.4,0.0000,1.0000
SQ,2.56729,8.51036,215.29,214.22,39.6073,neutral,True,432.8,0.0000,1.0000
MU,1.26827,18.12126,77.62,77.64,39.4032,buy,True,1113,0.0000,1.0000
ISR,2.27569,19.66579,1.7199,1.7199,39.2617,buy,True,68.9,NaN,NaN
CGC,2.37242,1.44393,39.95,40.41,38.2949,strong buy,True,350.11,0.0000,1.0000
DFS,1.75996,3.40184,83.625,83.09,36.8049,buy,True,307,0.5000,0.5000
AVGO,1.05056,1.55795,451.91,449.11,36.0359,buy,True,407,0.0000,1.0000
CURLF,1.57790,0.97750,13.3,13.6,35.7034,buy,True,470.09,0.0000,1.0000
LUV,1.19368,6.19623,44.1,43.88,34.0292,buy,True,590.47,0.6924,0.3076
AMAT,1.50476,8.33117,97.115,97,33.4622,buy,True,914,0.5000,0.5000


,beta,volume_10_day,c,o,adx,signal,trending,shareOutstanding,bearish,bullish
DD,1.66657,17.38543,78.81,78.585,31.6296,buy,True,738.56,0.0000,1.0000
BAC,1.55149,54.45254,29.84,29.695,31.0538,buy,True,8650.81,0.0000,1.0000
RAD,1.07665,2.57636,27.02,27.18,29.3905,strong buy,True,54.72,0.5000,0.5000
GS,1.49450,3.63392,272.66,271.319,28.6194,buy,True,344.07,0.7143,0.2857
CMG,1.31624,0.26732,1481.31,1477.73,28.49,buy,True,27.75,0.0000,1.0000
MS,1.52019,12.95171,67.325,67,28.4809,buy,True,1810,0.1667,0.8333
JACK,1.74193,0.29610,94.8,94.595,28.2699,buy,True,22.72,0.5000,0.5000
GOOS,1.54667,0.34385,33.54,33.4,28.2147,buy,True,110,1.0000,0.0000
MRO,3.36932,30.46996,7.26,7.3,27.7946,buy,True,796,0.0000,1.0000
JBLU,1.59672,7.80243,14.335,14.34,26.7937,buy,True,0,0.0000,1.0000


,beta,volume_10_day,c,o,adx,signal,trending,shareOutstanding,bearish,bullish
HAL,2.88349,12.78344,17.745,17.78,22.5673,buy,False,884.01,0.0000,1.0000
XOM,1.41141,30.80865,44.8,44.817,22.504,buy,False,4234,0.3000,0.7000
GDOT,1.02694,0.77347,49.99,49.92,22.0643,neutral,False,51.81,0.0000,1.0000
SIRI,1.10284,28.68150,6.305,6.305,21.8987,buy,False,4412,0.6000,0.4000
AMP,1.69999,0.64194,199.52,198.415,21.0186,buy,False,0,0.0000,1.0000
LOW,1.37086,4.04351,167.8,167.005,20.466,buy,False,763,0.0000,1.0000
COP,1.83843,15.14069,40.2,40.29,20.456,neutral,False,1084.87,0.5000,0.5000
PSX,1.70667,3.00369,67.92,68.08,20.4157,neutral,False,441.03,0.5000,0.5000
MAR,1.81733,2.28427,116.59,116.54,18.9809,buy,False,324,0.0000,1.0000
BA,1.59932,11.63311,194.68,194.523,18.9656,neutral,False,582.32,0.8462,0.1538


In [ ]:
# for DEV
# pattern recognition
# for stock in data_1.index:
#     r = requests.get('https://finnhub.io/api/v1/scan/pattern?symbol=' + str(stock) + '&resolution=D&token=bvt5fmf48v6rku8bvmn0')
#     points = r.json()["points"]
#     for entry in points:
#         display(entry)